### Randomly Generating matrix and kernals

In [255]:
import numpy as np
import random

A_VALUE_MAX = 4
A_VALUE_MIN = 0

B_VALUE_MAX = 8
B_VALUE_MIN = 0

MAT_SIZE = 16
KER_SIZE = 3
DRAM_STARTING_ADDRESS = 0x1000

DRAM1_END_ADDRESS = 0x1fff

DRAM_READ_INSTR_END_ADDRESS = 0x1fff

DRAM_READ_MAT_START_ADDRESS = 0x2000
DRAM_READ_MAT_END_ADDRESS = 0x2fff

# mat = np.array([[rd.randint(0,PRIME-1) for i in range(size)] for j in range(size)], dtype=object)


In [256]:
mat    = np.array([[random.randint(A_VALUE_MIN,A_VALUE_MAX) for i in range(MAT_SIZE)] \
                    for j in range(MAT_SIZE)],dtype=object)

kernal = np.array([[random.randint(A_VALUE_MIN,A_VALUE_MAX) for i in range(KER_SIZE)] \
                    for j in range(KER_SIZE)],dtype=object)

In [257]:
kernal

array([[4, 2, 4],
       [1, 2, 2],
       [4, 4, 2]], dtype=object)

In [258]:
PAT_NUM = (0x1fff - 0x1000)/4

### Generating address for DRAM1 used for storing 32-bit matrixA data

In [259]:
f_DRAM1 = open("./DRAM1.dat", "w")

In [260]:
for addr in range(DRAM_STARTING_ADDRESS,DRAM1_END_ADDRESS,4):
    f_DRAM1.write('@' + format(addr, 'x') + '\n')

    # 32-BIT data, {: > } means right-alignement for this temp string
    temp = '{:0>4d}'.format(random.randint(A_VALUE_MIN, A_VALUE_MAX), 'x')
    # print(temp)
    f_DRAM1.write( temp[3] + ' ' + temp[2] + ' ' + temp[1] + ' ' + temp[0] + '\n' )


f_DRAM1.close()

In [261]:
temp

'0004'

### Generating address and data for DRAM_read instruction and MatrixB

In [262]:
f_DRAM_read = open("./DRAM.dat", "w")

### Specially note that the starting position of your matrixA and matrixB is randomly chosen as long as they are within the addressable range

- Why isnt the data stored in the multiple of 4? That is making it only byte addressable?

In [263]:
for addr in range(DRAM_STARTING_ADDRESS,DRAM_READ_INSTR_END_ADDRESS,4):
    f_DRAM_read.write('@' + format(addr, 'x') + '\n')

    # Instruction generation [31:18] matrixA starting address for DRAM1
    # Note the generated address must be a multiple of 4
    while True:
        #Note the starting address of MatrixA must start from an address which contains all the 
        #16x16 32bits value, so the upper limit must be constrained by subtracting 16*16*4
        addr = random.randint(int(DRAM_STARTING_ADDRESS),int(DRAM1_END_ADDRESS - 16*16*4))
        # The starting address of MatrixA and MatrixB can be anywhere, as long as it is vali
        # It does not reall has to be a multiple of 4
        if addr % 16 == 0 or addr % 16 == 4 or addr % 16 == 8 or addr % 16 == 12 or addr % 3\
            or addr % 7 or addr % 9:
            break
    
    dram1_matrixA_addr = addr
    # Since upper instruction has the format [15:2](Starting address of DRAM_1) [1:0] (N/A)
    # Thus first times 4 then convert it into bit string format of 16 bit long
    instr_upper_16_bin = f'{addr * 4 :0>16b}'
    
    # [15:2] is the matrix starting address for DRAM_read
    while True:
        addr = random.randint(int(DRAM_READ_MAT_START_ADDRESS),int(DRAM_READ_MAT_END_ADDRESS-16*16*4))
        if addr % 16 == 0 or addr % 16 == 4 or addr % 16 == 8 or addr % 16 == 12 \
        or addr % 16 == 3 or addr % 16 == 7 or addr % 16 == 11 or addr % 16 == 15:
            break
    
    dram_read_matrixB_addr_bin = f'{addr:0>14b}' # conver into 14 bits representation

    # [1:0] as operating mode
    while True:
        op = random.randint(0,3)
        # 0 stands for matrix multiplication, 3 stands for convolution
        if op == 0 or op == 3:
            break
    
    dram_read_op_bin = f'{op:0>2b}'

    # Concatenate the MatrixB starting address with op code in binary
    instr_lower_16_bin = dram_read_matrixB_addr_bin + dram_read_op_bin

    instr_bin = instr_upper_16_bin + instr_lower_16_bin

    # This is the format of converting a binary format into hex format which is used extensively
    instr_hex = hex(int(instr_bin,2))

    # Note in the hex string, to select 2 char you must specify [8:10] which would selects [8] [9] of
    # the hex string
    f_DRAM_read.write(instr_hex[8:10] + ' ' + instr_hex[6:8] + ' ' + \
                       instr_hex[4:6] + ' '+ instr_hex[2:4])
    f_DRAM_read.write('\n')

for addr in range(DRAM_READ_MAT_START_ADDRESS,DRAM_READ_MAT_END_ADDRESS,4):
    f_DRAM_read.write('@' + format(addr, 'x') + '\n')

    # 32-BIT data, {: > } means right-alignement for this temp string
    temp = '{:0>4d}'.format(random.randint(B_VALUE_MIN, B_VALUE_MAX), 'x')
    # print(temp)
    f_DRAM_read.write( temp[3] + ' ' + temp[2] + ' ' + temp[1] + ' ' + temp[0] + '\n' )


f_DRAM_read.close()
# print(f"instr_hex : {instr_hex}")
instr_hex

'0x493080bf'

In [264]:
dram1_matrixA_addr

4684

In [265]:
dram_read_matrixB_addr_bin

'10000000101111'

In [266]:
dram_read_op_bin

'11'

# Notes

### Address location

- This is for writing address location, where 'x' means converting the addr integer into hex format
```python

    f_DRAM_read.write('@' + format(addr, 'x') + '\n')

```